In [23]:
import bibtexparser

with open('export-bibtex.bib') as bibtex_file:
    bib_database = bibtexparser.bparser.BibTexParser(common_strings=True).parse_file(bibtex_file)

In [24]:
bib_database.entries[1]

{'adsnote': 'Provided by the SAO/NASA Astrophysics Data System',
 'adsurl': 'https://ui.adsabs.harvard.edu/abs/2022AJ....163..144G',
 'doi': '10.3847/1538-3881/ac503c',
 'pages': '144',
 'eid': '144',
 'number': '4',
 'volume': '163',
 'month': 'April',
 'year': '2022',
 'keywords': '1761, 1213, 1681, 1290, 982, 1629, 1572, 1580, 1603',
 'journal': '\\aj',
 'title': '{Complex Modulation of Rapidly Rotating Young M Dwarfs: Adding Pieces to the Puzzle}',
 'author': '{G{\\"u}nther}, Maximilian N. and {Berardo}, David A. and {Ducrot}, Elsa and {Murray}, Catriona A. and {Stassun}, Keivan G. and {Olah}, Katalin and {Bouma}, L.~G. and {Rappaport}, Saul and {Winn}, Joshua N. and {Feinstein}, Adina D. and {Matthews}, Elisabeth C. and {Sebastian}, Daniel and {Rackham}, Benjamin V. and {Seli}, B{\\\'a}lint and {Triaud}, Amaury H.~M.~J. and {Gillen}, Edward and {Levine}, Alan M. and {Demory}, Brice-Olivier and {Gillon}, Micha{\\"e}l and {Queloz}, Didier and {Ricker}, George R. and {Vanderspek}, Ro

In [30]:
txts = []

def rem_braces(t):
    return t.replace("{", "").replace("}", "").replace("\pi", "$\pi$")

for paper in bib_database.entries:
    authors = rem_braces(paper["author"])
    authors = ", ".join(authors.split(" and ")[0:3]) + " et al."
    if "Garcia" not in authors:
        authors += " (including Garcia, L. J.)"
    else:
        authors.replace("Garcia, Lionel", "\\textbf{Garcia, Lionel}")
        
    title = rem_braces(paper["title"])
    date = rem_braces(paper["year"])
    if "publisher" in paper:
        journal = rem_braces(paper["publisher"])
    elif "journal" in paper:
        journal = f"{rem_braces(paper['journal'])}"
    else:
        continue

    if "VizieR" in journal:
        continue

    url = rem_braces(paper.get("adsurl", ""))

    if "volume" in paper:
        journal += f", {rem_braces(paper['volume'])}"
    if "number" in paper:
        journal += f", {rem_braces(paper['number'])}"
    if "page" in paper:
        journal += f", {rem_braces(paper['page'])}"

    final = "\\" + f"href{{{url}}}{{{journal}}}"

    txts.append(f"""\\publi{{{date}}}
    {{{title}}}
    {{{authors}}}
    {{{final}}}""")
    
open("works2.txt", "w").write("\n\n".join(txts))

2653

In [12]:
txts

['\\publi{2022}\n    {HST/WFC3 transmission spectroscopy of the cold rocky planet TRAPPIST-1h}\n    {Moran, Sarah, Rackham, Benjamin, Wakeford, Hannah et al. (including Garcia, L. J.)}\n    {}',
 '\\publi{2022}\n    {Complex Modulation of Rapidly Rotating Young M Dwarfs: Adding Pieces to the Puzzle}\n    {Berardo, David A., Ducrot, Elsa, Murray, Catriona A. et al. (including Garcia, L. J.)}\n    {\\aj}',
 '\\publi{2022}\n    {HST/WFC3 transmission spectroscopy of the cold rocky planet TRAPPIST-1h}\n    {Moran, S.~E., Rackham, B.~V., Wakeford, H.~R. et al. (including Garcia, L. J.)}\n    {arXiv e-prints}',
 '\\publi{2022}\n    {PROSE: a PYTHON framework for modular astronomical images processing}\n    {Timmermans, Mathilde, Pozuelos, Francisco J., Ducrot, Elsa et al. (including Garcia, L. J.)}\n    {\\mnras}',
 '\\publi{2021}\n    {Six transiting planets and a chain of Laplace resonances in TOI-178}\n    {Alibert, Y., Hara, N.~C., Hooton, M.~J. et al. (including Garcia, L. J.)}\n    {\\

In [18]:
paper

{'journal': 'Astronomy {\\&} Astrophysics',
 'title': '{SPECULOOS}: Ultracool dwarf transit survey',
 'author': 'D. Sebastian and M. Gillon and E. Ducrot and F. J. Pozuelos and L. J. Garcia and M. N. GÃ¼nther and L. Delrez and D. Queloz and B. O. Demory and A. H. M. J. Triaud and A. Burgasser and J. de Wit and A. Burdanov and G. Dransfield and E. Jehin and J. McCormac and C. A. Murray and P. Niraula and P. P. Pedersen and B. V. Rackham and S. Sohy and S. Thompson and V. Van Grootel',
 'pages': 'A100',
 'volume': '645',
 'publisher': '{EDP} Sciences',
 'month': 'jan',
 'year': '2021',
 'url': 'https://doi.org/10.1051%2F0004-6361%2F202038827',
 'doi': '10.1051/0004-6361/202038827',
 'ENTRYTYPE': 'article',
 'ID': 'Sebastian_2021'}

In [73]:
ads.config.token = '4Wb11yxAAyjXjffgcGsrLmopqA6rdagUwuNtDhTA'
papers = list(
    ads.SearchQuery(
        author="Garcia, Lionel J.",
        fl=[
            "id",
            "title",
            "author",
            "doi",
            "year",
            "pubdate",
            "pub",
            "volume",
            "page",
            "identifier",
            "doctype",
            "citation_count",
            "bibcode",
            "citation",
        ],
        max_pages=200,
    )
)

In [74]:
for paper in papers:
    if "planet" in paper.title:
        print(paper.title)

In [2]:
def get_papers(author, count_cites=True):
    papers = list(
        ads.SearchQuery(
            author=author,
            fl=[
                "id",
                "title",
                "author",
                "doi",
                "year",
                "pubdate",
                "pub",
                "volume",
                "page",
                "identifier",
                "doctype",
                "citation_count",
                "bibcode",
                "citation",
            ],
            max_pages=100,
        )
    )
    dicts = []

    # Count the citations as a function of time
    citedates = []

    # Save bibcodes for later
    bibcodes = []

    for paper in papers:

        if not (
            ("Luger, Rodrigo" in paper.author)
            or ("Luger, R." in paper.author)
            or ("Luger, R" in paper.author)
        ):
            continue

        if manual_exclude(paper):
            continue

        aid = [
            ":".join(t.split(":")[1:])
            for t in paper.identifier
            if t.startswith("arXiv:")
        ]
        for t in paper.identifier:
            if len(t.split(".")) != 2:
                continue
            try:
                list(map(int, t.split(".")))
            except ValueError:
                pass
            else:
                aid.append(t)
        try:
            page = int(paper.page[0])
        except ValueError:
            page = None
            if paper.page[0].startswith("arXiv:"):
                aid.append(":".join(paper.page[0].split(":")[1:]))
        except TypeError:
            page = None

        # Get citation dates
        if count_cites and paper.citation is not None:
            for i, bibcode in tqdm(
                enumerate(paper.citation), total=len(paper.citation)
            ):
                try:
                    cite = list(ads.SearchQuery(bibcode=bibcode, fl=["pubdate"]))[0]
                    date = int(cite.pubdate[:4]) + int(cite.pubdate[5:7]) / 12.0
                    citedates.append(date)
                except IndexError:
                    pass

        # Save bibcode
        bibcodes.append(paper.bibcode)

        dicts.append(
            dict(
                doctype=paper.doctype,
                authors=format_authors(paper.author),
                year=paper.year,
                pubdate=paper.pubdate,
                doi=paper.doi[0] if paper.doi is not None else None,
                title=format_title(paper.title[0]),
                pub=paper.pub,
                volume=paper.volume,
                page=page,
                arxiv=aid[0] if len(aid) else None,
                citations=paper.citation_count,
                url="http://adsabs.harvard.edu/abs/" + paper.bibcode,
            )
        )

    if count_cites:
        # Sort the cite dates
        citedates = sorted(citedates)
        np.savetxt("citedates.txt", citedates, fmt="%.3f")

    # Save bibcodes
    with open("bibcodes.txt", "w") as f:
        for bibcode in bibcodes:
            print(bibcode, file=f)

    return sorted(dicts, key=itemgetter("pubdate"), reverse=True)


In [39]:
for r in s.get():
    print(r.title, r.authors)

Direct laser writing for cardiac tissue engineering: a microfluidic heart on a chip with integrated transducers [<arxiv.arxiv.Result.Author object at 0x108f85e10>, <arxiv.arxiv.Result.Author object at 0x108fab710>, <arxiv.arxiv.Result.Author object at 0x108fab350>, <arxiv.arxiv.Result.Author object at 0x108fab390>, <arxiv.arxiv.Result.Author object at 0x108fab450>, <arxiv.arxiv.Result.Author object at 0x108f9ca50>, <arxiv.arxiv.Result.Author object at 0x108faac90>, <arxiv.arxiv.Result.Author object at 0x108f96d50>, <arxiv.arxiv.Result.Author object at 0x108fbd190>]
The XFaster Power Spectrum and Likelihood Estimator for the Analysis of Cosmic Microwave Background Maps [<arxiv.arxiv.Result.Author object at 0x108ede710>, <arxiv.arxiv.Result.Author object at 0x108f85e50>, <arxiv.arxiv.Result.Author object at 0x108fa2c50>, <arxiv.arxiv.Result.Author object at 0x108fa2150>, <arxiv.arxiv.Result.Author object at 0x108fa2390>, <arxiv.arxiv.Result.Author object at 0x108fa2a50>, <arxiv.arxiv.Res

In [18]:
list(res)

[]

In [34]:
import urllib
url = 'http://export.arxiv.org/api/query?search_query=all:electron&start=0&max_results=1'
data = urllib.urlopen(url).read()
print(data)

AttributeError: module 'urllib' has no attribute 'urlopen'